In [ ]:
# importing essential libraries 
%load_ext google.colab.data_table
import pandas as pd
import numpy as np
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from fbprophet import Prophet
import itertools
from datetime import datetime
from fbprophet.plot import plot_yearly
import warnings
import seaborn as sns
from itertools import cycle

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
!ls "/content/gdrive/My Drive/team-project/data/m5-forecasting-accuracy"

In [ ]:
# defining data path
datadir="/content/gdrive/My Drive/team-project/data/m5-forecasting-accuracy/"

In [ ]:
# reading data into panda dataframes
calendar = pd.read_csv(f'{datadir}/calendar.csv')
selling_prices = pd.read_csv(f'{datadir}/sell_prices.csv')
sample_submission = pd.read_csv(f'{datadir}/sample_submission.csv')
sales_train_val = pd.read_csv(f'{datadir}/sales_train_validation.csv')
sales_eval = pd.read_csv(f'{datadir}/sales_train_evaluation.csv')

In [ ]:
calendar.head()


In [ ]:
calendar.shape

In [ ]:
selling_prices.head()

In [ ]:
sales_train_val.head()


In [ ]:
sales_eval.head()


In [ ]:
columns = sales_eval.columns
date_columns = columns[columns.str.contains("d_")]
dates_s = [pd.to_datetime(calendar.loc[calendar['d'] == str_date,'date'].values[0]) for str_date in date_columns]

In [ ]:
sales_eval = sales_eval.drop(['item_id','dept_id','cat_id','store_id','state_id'], axis=1)
valid_cols = ['d_'+str(1913+i) for i in range(1,29)]
sales_eval.columns = sales_eval.columns.tolist()[:-28]+valid_cols
sales_eval.columns

In [ ]:
sales_eval.tail()

In [ ]:
item1 = sales_eval.iloc[0]

In [ ]:
item1

In [ ]:
item1 = item1.drop('id').T.reset_index().merge(calendar[['d','date']], left_on='index', right_on='d', how='left').drop(['index','d'], axis=1)

In [ ]:
item1.head()

In [ ]:
item1.shape

In [ ]:
# modeling and visualizing on one row
item = sales_eval.iloc[0]
item = item.drop('id').T.reset_index().merge(calendar[['d','date']], left_on='index', right_on='d', how='left').drop(['index','d'], axis=1)
item.columns = ['y', 'ds']
item.y = item.y.astype('float')
item.ds = item.ds.astype('datetime64')
train_item = item.iloc[:-28]
valid_item = item.iloc[-28:]

# defining parameters for proophet model
gr ='linear'
sm ='multiplicative' 
iw = 0.8
ds = False 
ys = True 
ws = True 
fom = 20
p = 10
ph = Prophet(growth= gr,
            seasonality_mode= sm,
            interval_width = iw,
            daily_seasonality= ds, 
            yearly_seasonality= ys,
            weekly_seasonality= ws).add_seasonality(
                            name='mensual', period=30.5,fourier_order=fom)

ph.fit(train_item)
forecast = ph.predict(valid_itemtem)
figure = ph.plot(forecast)
figure.show()

In [ ]:
ph.plot_components(forecast)

In [ ]:
forecast.head()

In [ ]:
forecast[['ds', 'yhat','trend','weekly','yearly']]

In [ ]:
predicted = forecast['yhat']


In [ ]:
# defining error metrics

def find_error(actual, pred,algo):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    print("{} RMS error:{}".format(algo,rmse))

    r2 = r2_score(actual, pred)
    print("{} R2 score:{}".format(algo,r2))
    return rmse, r2

In [ ]:
rmse = find_error(item['y'],predicted,'prophet')

In [ ]:
# modeling and visualizing  for all rows
for i in range(len(sales_eval)):
  item = sales_eval.iloc[i]
  item_id = item.id
  item = item.drop('id').T.reset_index().merge(calendar[['d','date']], left_on='index', right_on='d', how='left').drop(['index','d'], axis=1)
  item.columns = ['y', 'ds']
  item.y = item.y.astype('float')
  item.ds = item.ds.astype('datetime64')
  train_item = item.iloc[:-28]
  valid_item = item.iloc[-28:]

  gr ='linear' 
  sm ='multiplicative'
  iw = 0.8
  ds = False
  ys = True
  ws = True 
  fom = 20 
  p = 10 
  ph = Prophet(growth= gr,
                seasonality_mode= sm,interval_width = iw,daily_seasonality= ds, yearly_seasonality= ys,weekly_seasonality= ws).add_seasonality(name='mensual', period=30.5,fourier_order=fom)
                
  ph.fit(train_item)
  forecast = ph.predict(item[['ds']])

In [ ]:
predicted1=forecast['yhat']
predicted1

In [ ]:
#evaluation 
rmse1 = find_error(item['y'],predicted1,'prophet')